# This notebook gives an overview of how to interact with the mongo database for the ophys_glm

dougo  
8/7/2020

Note that the database is hosted at `ibs-dougo-ux1.corp.alleninstitute.org:9999` That address is subject to change. However, I have some convenience functions that deal with connections, addresses, etc., so you don't need to keep track of the actual IP/port of the DB.

## Getting what you need using GLM_analysis_tools
I've built some convenience functions to access the tables holding visual_behavior_glm info. Here's how to use them

### convenience functions are in `GLM_analysis_tools`, import it

In [1]:
import visual_behavior_glm.src.GLM_analysis_tools as gat

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### access the full results
the `results_full` collection holds `avg_cv_var` records for every experiment/cell/version  

To access it, use the `retrieve_results()` method in `GLM_analysis_tools` and pass the `results_type='full'` argument

You can also pass a `search_dict` to narrow results to a specific set of search keys. By default, the full table will be returned, which can take 20-30 seconds.  

NOTE: this table contains a lot of metadata for every experiment. I've realized that this is unnecessary. It both slows queries and wastes database space. I will remove these superfulous fields at some point and we can simply merge with the ophys_experiment_table when we need them.

In the example below, we'll get all results for a given OEID, which will include results for every version

In [2]:
results = gat.retrieve_results({'ophys_experiment_id':808621958}, results_type='full')
results

,_id,cell_specimen_id,Full_avg_cv_var_train,Full_avg_cv_var_test,intercept_avg_cv_var_train,intercept_avg_cv_var_test,time_avg_cv_var_train,time_avg_cv_var_test,pre_licks_avg_cv_var_train,pre_licks_avg_cv_var_test,post_licks_avg_cv_var_train,post_licks_avg_cv_var_test,pre_lick_bouts_avg_cv_var_train,pre_lick_bouts_avg_cv_var_test,post_lick_bouts_avg_cv_var_train,post_lick_bouts_avg_cv_var_test,rewards_avg_cv_var_train,rewards_avg_cv_var_test,change_avg_cv_var_train,change_avg_cv_var_test,hits_avg_cv_var_train,hits_avg_cv_var_test,misses_avg_cv_var_train,misses_avg_cv_var_test,false_alarms_avg_cv_var_train,false_alarms_avg_cv_var_test,correct_rejects_avg_cv_var_train,correct_rejects_avg_cv_var_test,omissions_avg_cv_var_train,omissions_avg_cv_var_test,image_expectation_avg_cv_var_train,image_expectation_avg_cv_var_test,running_avg_cv_var_train,running_avg_cv_var_test,pupil_avg_cv_var_train,pupil_avg_cv_var_test,image0_avg_cv_var_train,image0_avg_cv_var_test,image1_avg_cv_var_train,image1_avg_cv_var_test,image2_avg_cv_var_train,image2_avg_cv_var_test,image3_avg_cv_var_train,image3_avg_cv_var_test,image4_avg_cv_var_train,image4_avg_cv_var_test,image5_avg_cv_var_train,image5_avg_cv_var_test,image6_avg_cv_var_train,image6_avg_cv_var_test,image7_avg_cv_var_train,image7_avg_cv_var_test,model_bias_avg_cv_var_train,model_bias_avg_cv_var_test,model_task0_avg_cv_var_train,model_task0_avg_cv_var_test,model_omissions1_avg_cv_var_train,model_omissions1_avg_cv_var_test,model_timing1D_avg_cv_var_train,model_timing1D_avg_cv_var_test,all-images_avg_cv_var_train,all-images_avg_cv_var_test,visual_avg_cv_var_train,visual_avg_cv_var_test,beh_model_avg_cv_var_train,beh_model_avg_cv_var_test,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,entry_time_utc,face_motion_energy_avg_cv_var_train,face_motion_energy_avg_cv_var_test
0,5f218aec952f973def1b981e,817103095,0.049649,0.030641,0.049589,0.030031,0.049636,0.030752,0.047840,0.031374,0.049378,0.030858,0.048359,0.031086,0.048913,0.029979,0.049172,0.030615,0.049381,0.030491,0.049300,0.030502,0.049533,0.030598,0.049637,0.030641,0.049577,0.030640,0.031505,0.013797,0.048548,0.029706,0.046002,0.028504,0.047906,0.032125,0.049098,0.030319,0.049144,0.030341,0.049340,0.030429,0.049278,0.030318,0.048810,0.030254,0.049207,0.030563,0.048626,0.030032,0.048218,0.029518,0.049528,0.031057,0.049509,0.032064,0.049571,0.031659,0.048943,0.031377,0.045262,0.028883,0.025742,0.010718,0.048200,0.032833,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,3,2020-07-30 04:00:35.518562,NaN,NaN
1,5f218aec952f973def1b981f,817102140,0.116994,-0.041268,0.116195,0.005533,0.116879,-0.040365,0.114151,-0.050217,0.116864,-0.042321,0.115800,-0.040319,0.116369,-0.042193,0.116624,-0.040558,0.116672,-0.041772,0.116923,-0.041113,0.116778,-0.041664,0.116987,-0.041268,0.116969,-0.041255,0.092792,-0.073389,0.116750,-0.040984,0.101803,-0.041329,0.113034,-0.068476,0.116552,-0.042834,0.116722,-0.035591,0.116847,-0.041713,0.116213,-0.036138,0.116578,-0.043062,0.116729,-0.040594,0.115799,-0.043396,0.115906,-0.044299,0.115916,-0.038229,0.115056,0.081890,0.115825,0.030527,0.113504,-0.068465,0.112693,-0.038971,0.087431,-0.071496,0.109774,0.094130,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.00000

### access the dropout summary
the `results_summary` collection holds the dropout summary, or the calculated change in variance explained for every experiment/cell/version/dropout, as comparied to the `Full` model for every experiment/cell/version.

To access it, use the `retrieve_results()` method in `GLM_analysis_tools` and pass the `results_type='summary'` argument

You can also pass a `search_dict` to narrow results to a specific set of search keys. By default, the full table will be returned, which can take a couple of minutes (it's currently at 12 million rows).  

NOTE: this table contains a lot of metadata for every experiment. I've realized that this is unnecessary. It both slows queries and wastes database space. I will remove these superfulous fields at some point and we can simply merge with the ophys_experiment_table when we need them.

In the example below, we'll get all results for a given OEID and a given version.

In [3]:
dropout_summary = gat.retrieve_results(
    {
        'ophys_experiment_id':808621958, 
        'glm_version':'5_L2_optimize_by_session'
    }, 
    results_type='summary'
)
dropout_summary

,_id,index,dropout,variance_explained,fraction_change_from_full,absolute_change_from_full,cell_specimen_id,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,glm_version,entry_time_utc
0,5f2920dba9fcc02f7f72683c,0,Full,-0.065943,-0.000000,0.000000,817102078,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.336335
1,5f2920dba9fcc02f7f72685a,30,all-images,-0.061120,-0.073144,0.004823,817102078,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.404717
2,5f2920dba9fcc02f7f72685c,32,beh_model,-0.071371,0.082317,-0.005428,817102078,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.408841
3,5f2920dba9fcc02f7f726844,8,change,-0.065985,0.000636,-0.000042,817102078,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.354252
4,5f2920dba9fcc02f7f726848,12,correct_rejects,-0.065846,-0.001471,0.000097,817102078,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.363534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,5f2920dba9fcc02f7f72680a,16,pupil,-0.027649,-0.016432,0.000462,817103784,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.222715
1481,5f2920dba9fcc02f7f726801,7,rewards,-0.028097,-0.000508,0.000014,817103784,808621958,808092249,808107647,814797528,VisualBehavior,container_qc,passed,20190114_421136_6imagesB,OPHYS_6_images_B,CAM2P.5,2019-01-14 16:06:55.000000,767468166,760949544,M,137.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,760949544,Vip-IRES-Cre,NaN,NaN,1,1,Vip_VISp_175,5_L2_optimize_by_session,2020-08-04 08:48:27.202454
1482,5f2920dba9fcc02f7f726

### access the weights
the `weight_matrix_lookup_table` collection holds the `w_matrix_id` for the W matrix for every experiment/version. 

Since the W matrix is an xarray, the actual values are stored in a separate part of the database using the `xarray_mongodb` library (`pip install xarray-mongodb`). This library deals with all of the under-the-hood mechanics of storing/retrieving xarrays in mongo, including splitting the arrays across multiple documents to avoid the mongo max document size limit. Bottom line is that we should only use the convenience functions below for storing/accessing the W arrays.

To access them, use the `get_weights_matrix_from_mongo()` method in `GLM_analysis_tools`. `ophys_experiment_id` and `glm_version` are required arguments. So you can only get one W matrix at a time.

See the `log_weights_matrix_to_mongo()` method in GLM_analysis_tools to see how the weights matrices are written to mongo.

In the example below, we'll get the W matrix for a given OEID and a given version.

In [4]:
W = gat.get_weights_matrix_from_mongo(ophys_experiment_id=808621958, glm_version='5_L2_optimize_by_session')
W

<xarray.DataArray (weights: 4631, cell_specimen_id: 45)>
array([[ 0.0070872 , -0.08006248,  0.01410601, ...,  0.01453127,
        -0.00027561,  0.00191877],
       [ 0.00696827,  0.04384637,  0.02147145, ...,  0.01832955,
        -0.00020025,  0.02898319],
       [-0.00577122, -0.00540147, -0.00486432, ..., -0.00823261,
        -0.00404761,  0.00017176],
       ...,
       [ 0.00256697,  0.00620962,  0.01256823, ...,  0.00543324,
         0.00258643, -0.00027064],
       [ 0.00268351,  0.00738718,  0.01507217, ...,  0.00662582,
         0.00310551, -0.00031405],
       [ 0.00292369,  0.00864965,  0.0175433 , ...,  0.00787313,
         0.00332674, -0.00038692]])
Coordinates:
  * weights           (weights) <U21 'intercept_0' ... 'model_timing1D_7'
  * cell_specimen_id  (cell_specimen_id) int64 817103095 817102140 ... 817102877

### access the error logs
The `error_logs` collection contains entries for every error we have logged. So far, this only includes kernel errors due to incomplete data streams, but could contain more in the future.

In the example below, we'll get the full error log table. But it's also possible to search by any key by passing a `search_dict` parameter with key/value pairs for each search field

In [5]:
error_table = gat.get_error_log()
error_table

,_id,error_type,kernel_name,exception,oeid,glm_version,entry_time_utc
0,5f2b27f382cfcf712604ed7f,kernel,pupil,Error! The number of sync file frame times (27...,940354181,5_L2_fixed_lambda=1,2020-08-05 21:43:15.072967
1,5f2b369155fd6ef797998b99,kernel,face_motion_PC_0,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.365544
2,5f2b369155fd6ef797998b9b,kernel,face_motion_PC_1,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.381381
3,5f2b369155fd6ef797998b9d,kernel,face_motion_PC_2,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.395898
4,5f2b369155fd6ef797998b9f,kernel,face_motion_PC_3,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.410134
5,5f2b369155fd6ef797998ba1,kernel,face_motion_PC_4,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.424880
6,5f2b369155fd6ef797998ba3,kernel,face_motion_PC_5,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.439574
7,5f2b369155fd6ef797998ba5,kernel,face_motion_PC_6,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.453703
8,5f2b369155fd6ef797998ba7,kernel,face_motion_PC_7,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.468151
9,5f2b369155fd6ef797998ba9,kernel,face_motion_PC_8,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.482546


## More general mongo stuff

the basic structure of mongo is:
* Document - a single entry, which is stored in a format similar to JSON (actually BSON, which is a binary form of JSON). I generally think of these as Python dictionaries, and I have some convenience functions for ensuring that dictionaries can write as mongo documents. Also note that every document has a unique `_id` assigned to it, which is a BSON.ObjectID object. You can search on this field if you'd like, and collections are automatically indexed by this value.
* Collection - a collection of documents. Somewhat analagous to a table in a standard relational datbase. We currently have collections titled `results_full`, `results_summary`, `weight_matrix_lookup_table`, and `error_logs`. The main difference from a standard RDB is that every document in a collection need not be standardized. But it's good practice to make them as similar as possible.
* Database - can contain multiple collections. We are using a database called `ophys_glm`

### To access mongo directly
I've written a Database class in visual_behavior.database that is built on top of pymongo. I'll demonstrate some basic useage to help explain how to generate new collections and/or access data without the convenience functions above.

#### First, import the class

In [6]:
import visual_behavior.database as database

#### Then establish a connection. 
I've called this server 'visual_behavior_data' and I have a text file on the network with connection details so you don't have to know them.

In [7]:
conn = database.Database('visual_behavior_data')

#### connect to the specific database we're using for this project (`ophys_glm`)

In [8]:
db = conn['ophys_glm']

#### view the collection names

In [9]:
db.list_collection_names()

['error_logs', 'results_summary', 'results_full', 'weight_matrix_lookup_table']

#### search for documents inside a given collection
the `find` function searches for matches using key/value pairs in a passed dictionary
the result is a cursor object

In [10]:
result = db['error_logs'].find({'oeid':855582961, 'kernel_name':'face_motion_PC_2'})
result

#### to actually view the result, it needs to be cast to a list
In this example, the list is only one entry long, but it will generally be one entry per matching result

In [11]:
list(result)

[{'_id': ObjectId('5f2b369155fd6ef797998b9d'),
  'error_type': 'kernel',
  'kernel_name': 'face_motion_PC_2',
  'exception': "'BehaviorOphysDataset' object has no attribute 'behavior_movie_timestamps'",
  'oeid': 855582961,
  'glm_version': 1,
  'entry_time_utc': '2020-08-07 16:53:25.395898'}]

#### I generally like to turn the results directly into a Pandas dataframe:

In [12]:
result = db['error_logs'].find({'oeid':855582961, 'kernel_name':'face_motion_PC_2'})
pd.DataFrame(list(result))

,_id,error_type,kernel_name,exception,oeid,glm_version,entry_time_utc
0,5f2b369155fd6ef797998b9d,kernel,face_motion_PC_2,'BehaviorOphysDataset' object has no attribute...,855582961,1,2020-08-07 16:53:25.395898


#### use find_one
There is also a `find_one` command, but I find that less useful since it will only return one result, even if there are multiple matches to your search.

The advantage is that the result is returned as a dictionary without the need to fiddle around with casting to a new type.

In [13]:
db['error_logs'].find_one({'oeid':855582961, 'kernel_name':'face_motion_PC_2'})

{'_id': ObjectId('5f2b369155fd6ef797998b9d'),
 'error_type': 'kernel',
 'kernel_name': 'face_motion_PC_2',
 'exception': "'BehaviorOphysDataset' object has no attribute 'behavior_movie_timestamps'",
 'oeid': 855582961,
 'glm_version': 1,
 'entry_time_utc': '2020-08-07 16:53:25.395898'}

#### create a new collection
We can create a new collection with a single line. Note that I'm first checking to see if the collection already exists to avoid an error

In [14]:
if 'temp_collection' not in db.list_collection_names():
    db.create_collection('temp_collection')

#### insert a document into our collection

In [15]:
document = {'title':'test','val_1':3,'val_2':1.5}
db['temp_collection'].insert_one(document)

#### now find the document we just inserted

In [16]:
db['temp_collection'].find_one({'title':'test'})

{'_id': ObjectId('5f2daf3185084bf02c690e13'),
 'title': 'test',
 'val_1': 3,
 'val_2': 1.5}

### Convenience Functions

#### `clean_and_timestamp`
Mongo is a little picky about data types. For example, you can't write a numpy.float64 object. Instead, we need plain python types (e.g. float). So I wrote a convenience function that will standardize the types in a dictionary, and also adds a UTC timestamp field as a bonus. I tend to pass my documents through this function before  writing to mongo.

In [21]:
import numpy as np
input_document = {
    'experiment_id':123456,
    'some_float':np.float64(np.pi),
    'some_int':np.int(3)
}
print(input_document)
print(type(input_document['some_float']))

{'experiment_id': 123456, 'some_float': 3.141592653589793, 'some_int': 3}
<class 'numpy.float64'>


note that after passing through this function, the type has changed and we now have a timestamp

In [22]:
cleaned_input_document = database.clean_and_timestamp(input_document)
print(cleaned_input_document)
print(type(cleaned_input_document['some_float']))

{'experiment_id': 123456, 'some_float': 3.141592653589793, 'some_int': 3, 'entry_time_utc': '2020-08-07 19:45:13.452201'}
<class 'float'>


#### `update_or_create`
Mongo has commands for creating new documents (`insert_one`, `insert_many`) and also for updating existing documents (`update_one`, `update_many`). But I've written a convenience function that will either update or create a document, depending on whether it already exists.

Inputs are:
* collection - the collection you want to write to.
* document - the document you want to write (should have already been converted to basic types, see above)
* keys_to_check - a list of the keys to use to determine if the document should be updated or created
* force_write - a boolean that, if True, will write a new document even if a matching one already exists.

In [23]:
database.update_or_create(
    collection=db['temp_collection'],
    document=cleaned_input_document,
    keys_to_check=['experiment_id']
)

#### now find our new document:

In [24]:
db['temp_collection'].find_one({'experiment_id':123456})

{'_id': ObjectId('5f2daf4b85084bf02c690e15'),
 'experiment_id': 123456,
 'some_float': 3.141592653589793,
 'some_int': 3,
 'entry_time_utc': '2020-08-07 19:45:13.452201'}

#### I'm going to delete our `temp_collection` Note that there are also commands for deleting documents within collectins. 

#### *_Be very careful with these commands - it'd be easy to inadvertently delete documents or entire collections._*

In [25]:
db.drop_collection('temp_collection')

{'ns': 'ophys_glm.temp_collection', 'nIndexesWas': 1, 'ok': 1.0}

#### close the connection when done
Good practice. Otherwise your connection will remain open until you end your python process.

In [26]:
conn.close()